In [ ]:
using Distributions
using StatsPlots
using StatsBase
using CSV
using DataFrames
using LinearAlgebra
using Random
using Turing

In [ ]:
# helper function for getting probability/credibility interval
CI(d; p = 0.95) = [quantile(d, (1-p)/2), quantile(d, (1+p)/2)]

In [ ]:
d = DataFrame(CSV.File("data/Howell1.csv"));
d2 = d[d.age .>= 18,:];  # select only those data from adults

In [ ]:
d2

In [ ]:
describe(d2)

In [ ]:
scatter(d2.weight, d2.height, xlabel="weight", ylabel="hegiht", legend=false)

In [ ]:
x̄ = mean(d2.weight)

In [ ]:
h̄ = mean(d2.height)

In [ ]:
xlims = extrema(d2.weight)  # getting min and max in one pass for later plotting purposes
ylims = extrema(d2.height)

In [ ]:
p = hline([0, 272]; ylims=(-100, 400), xlabel="weight", ylabel="hegiht")
scatter!(d2.weight, d2.height)

In [ ]:
# see what the normal distributions for α and β give
Random.seed!(2971)
N = 100
α = rand(Normal(178, 20), N);
β = rand(Normal(0, 10), N);

In [ ]:
for (α_, β_) ∈ zip(α, β)
    plot!(x -> α_ + β_ * (x - x̄); xlims=xlims, c=:black, alpha=0.3, legend=false)
end
title!("β ~ Normal(μ = 0, σ = 10)")
display(p)

In [ ]:
β = rand(LogNormal(0, 1), 10_000)
density(β, xlims=(0, 5), bandwidth=0.01)

In [ ]:
# see what the log-normal distribution for β gives

p = hline([0, 272]; ylims=(-100, 400), xlabel="weight", ylabel="hegiht")

title!("β ~ LogNormal(μ = 0, σ = 1)")

for (α_, β_) ∈ zip(α, β)
  plot!(x -> α_ + β_ * (x - x̄); xlims=xlims, c=:black, alpha=0.3, legend=false)
end

scatter!(d2.weight, d2.height)

display(p)


In [ ]:
# define our Bayesian statistics model for linear regression
@model function height_regr_model(weight, height)
    
    α ~ Normal(178, 20)
    β ~ LogNormal(0, 1)
    σ ~ Uniform(0, 50)
  
    for i in 1:length(weight)
        height[i] ~ Normal(α + β * (weight[i] - x̄), σ)
    end

    #μ = α .+ β * (weight .- x̄)
    #height ~ MvNormal(μ, σ)

end

In [ ]:
m = sample(height_regr_model(d2.weight, d2.height), NUTS(), 1000);

In [ ]:
# now take 1000 samples from the posterior distribution obtained from the MC method
samples = sample(m, 1000);

In [ ]:
DataFrame(samples)

In [ ]:
plot(samples)

In [ ]:
# Use the mean values of α and β to plot the "best" fit line
α_mean = mean(samples[:α])


In [ ]:
β_mean = mean(samples[:β])

In [ ]:
scatter(d2.weight, d2.height; alpha=0.3)
plot!(x -> α_mean + β_mean*(x-x̄))

In [ ]:
# Let's reduce the sample size to 10 to explore posterior distributions of α and β
N = 10
dN = d2[1:N,:]

In [ ]:
x̄N = mean(dN.weight)

In [ ]:
p = scatter(dN.weight, dN.height; xlims=xlims, ylims=ylims, label=false)
title!("N = $N"; xlab="weight", ylab="height")


In [ ]:
mN = sample(height_regr_model(dN.weight, dN.height), NUTS(), 1000);

In [ ]:
# sample 20 values of α and β from the posterior
samplesN = sample(mN, 20);
samplesN_df = DataFrame(samplesN)

In [ ]:
for (α_, β_) ∈ zip(samplesN_df.α, samplesN_df.β)
    plot!(x -> α_ + β_ * (x-x̄N); c="black", alpha=0.3, legend=false)
end
display(p)

In [ ]:
# Let's change the sample size to a different value to explore posterior distributions of α and β
N = 100
dN = d2[1:N,:];

x̄N = mean(dN.weight)

mN = sample(height_regr_model(dN.weight, dN.height), NUTS(), 1000);

# sample 20 values of α and β from the posterior
samplesN = sample(mN, 20);
samplesN_df = DataFrame(samplesN);

p = scatter(dN.weight, dN.height; xlims=xlims, ylims=ylims, label=false);
title!("N = $N"; xlab="weight", ylab="height");

x̄N = mean(dN.weight);
for (α_, β_) ∈ zip(samplesN_df.α, samplesN_df.β)
    plot!(x -> α_ + β_ * (x-x̄N); c="black", alpha=0.3, legend=false)
end
display(p)


In [ ]:
# Let's now go back to our original posterior distribution obtained from all the adult data and take 1000 samples
samples = sample(m, 1000);
samples_df = DataFrame(samples)


In [ ]:
# posterior distribution of μ for one particular value of weight
μ_at_50 =  samples_df.α .+ samples_df.β * (50 - x̄);
density(μ_at_50; lw=2, xlab="μ|weight=50")

In [ ]:
# now do the same for all weight values in the data

μ = zeros(1000, 352);

for i in 1:1000
    for j in 1:352
        μ[i,j] = samples_df.α[i] + samples_df.β[i] * (d2.weight[j] - x̄)
    end
end

μ

In [ ]:
# a fancier way to do the same
μ = map(w -> samples_df.α + samples_df.β * (w - x̄), d2.weight);

# 352-element vector where each element is a vector of 1000 elements
# these 1000 elements are obtained from 1000 combinations of α and β sampled from the posterior

μ = hcat(μ...) # turn the above array into 1000 x 352 matrix

In [ ]:
# plot the first 100 values of μ
p = plot()
for i in 1:100
    scatter!(d2.weight, μ[i,:]; c=:blue, alpha=0.2, label=false)
end
display(p)

In [ ]:
# Now, consider μ for a defined weight sequence (not the weight values from the data)
weight_seq = 25:70

In [ ]:
μ_seq = zeros(1000, length(weight_seq));

for i in 1:1000
    for j in 1:length(weight_seq)
        μ_seq[i,j] = samples_df.α[i] + samples_df.β[i] * (weight_seq[j] - x̄)
    end
end

μ_seq

In [ ]:
μ_seq_mean = zeros(length(weight_seq));
μ_seq_CI = zeros(length(weight_seq),2);

for j in 1:length(weight_seq)  # mean and credible interval for each column (each value of weight)
    μ_seq_mean[j] = mean(μ_seq[:,j])
    μ_seq_CI[j,:] = CI(μ_seq[:,j])
end

In [ ]:
# a fancier way to do the same
μ_seq = map(w -> samples_df.α + samples_df.β * (w - x̄), weight_seq)
μ_seq = hcat(μ_seq...)

μ_seq_mean = mean.(eachcol(μ_seq))
μ_seq_CI = CI.(eachcol(μ_seq))
μ_seq_CI = vcat(μ_seq_CI'...) # convert vector of vector into 46x2 matrix

In [ ]:
μ_seq_CI

In [ ]:
scatter(d2.weight, d2.height; alpha=0.2, xlab="weight", ylab="height")
plot!(weight_seq, μ_seq_mean; c=:black)
plot!(weight_seq, [μ_seq_mean μ_seq_mean]; c=:black, fillrange=μ_seq_CI, fillalpha=0.3, label=false)

In [ ]:
# We now do the similar simulation while including the effect of σ on the posterior distribution
sim_height = map(w -> rand.(Normal.(samples_df.α + samples_df.β * (w - x̄), samples_df.σ)), weight_seq)
sim_height = hcat(sim_height...);

height_CI = CI.(eachcol(sim_height))
height_CI = vcat(height_CI'...);

plot!(weight_seq, [μ_seq_mean μ_seq_mean]; c=:black, fillrange=height_CI, fillalpha=0.3)
